In [1]:
import os
import pandas as pd
import numpy as np
from utils import to_21d_vec, get_beat_vector
import re
import sys
sys.path.insert(1, './harmalysis')
from inputRN2Chord import inputRN

In [4]:
script_dir = os.getcwd()
score_rel_path = "../datasets/ABC/ABC_reduced_score_for_vec/n08op59-2_04_score_reduced4_tmp_A4_trans_vector_pre.krn"
scorepath = os.path.join(script_dir, score_rel_path)


In [5]:
df = pd.read_csv(scorepath, sep="\t", header=None)
pd.set_option('display.max_rows', df.shape[0]+1)
#df.columns = ["harm", "root", "voice4", "voice3", "voice2", "voice1", "index", "beat", "measure", "key", "meter" ]
#df.columns = ["harm", "root", "voice4", "voice3", "voice2", "voice1", "beat", "measure", "key", "meter" ]
#df.columns = ["harm", "voice4", "voice3", "voice2", "voice1", "index", "beat", "measure", "key", "meter" ]
#df.columns = ["harm", "voice4", "voice3", "voice2", "voice1", "beat", "measure", "key", "meter" ]

#sear org
#df.columns = ["voice4", "voice3", "voice2", "voice1", "harm", "beat", "measure", "key", "meter" ]

# abc org
# abc reduce
df.columns = ["voice4", "dy4", "voice3", "dy3", "voice2", "dy2", "voice1", "dy1",
              "harm", "index", "beat", "measure", "key", "meter" ]

In [6]:
# process dataframe, delete bars, score representations, and "rest slice"
df = df[~df['beat'].astype(str).str.startswith(('=','.','*'))]
df

,voice4,dy4,voice3,dy3,voice2,dy2,voice1,dy1,harm,index,beat,measure,key,meter
2,4FF#',fp,4A#',fp,4c#',fp,2.f#,fp,VI,13,1,=1,a#,M2/2
4,8F#',fp,8A#',fp,8c#',fp,2.f#,fp,VI,15,2.5,=1,a#,M2/2
5,4F#',fp,4A#',fp,4c#',fp,2.f#,fp,VI,16,3,=1,a#,M2/2
6,8r,fp,8r,fp,8r,fp,(8fff#L,fp,VI,17,4,=1,a#,M2/2
7,8F#',fp,8A#',fp,8c#',fp,8r,fp,VI,18,4.5,=1,a#,M2/2
9,4F#,fp,4A#,fp,4c#,fp,4aaa#'),fp,VI.I,20,1,=2,a#,M2/2
11,8F#,fp,8A#,fp,8c#,fp,8r,fp,VI.I,22,2.5,=2,a#,M2/2
12,4F#,fp,4B,fp,4d#,fp,4fff#',fp,I(64),23,3,=2,a#,M2/2
14,8F#,fp,8B,fp,8r,fp,8ddd#',fp,I(64),25,4.5,=2,a#,M2/2
16,4F#,fp,4A#,fp,4f#,fp,2.ccc#,fp,I,27,1,=3,a#,M2/2


In [5]:
prev_note_list = []
for index, row in df.iterrows():
    # for this onset slice 
    this_onset_vec = []
    
    ######################################### Process notes and onsets ########################################
    voices = ["voice4","voice3","voice2","voice1"]
    cur_note_tmp = []
    for v in voices:
        this_note = ''.join(row[[v]].values)
        if ' ' in this_note: #for a part has multiple voices
            this_note_list = this_note.split(" ")
            cur_note_tmp.extend(this_note_list)
        else:
            cur_note_tmp.append(this_note)
    
    cur_note_list = []
    for n in cur_note_tmp:
        note_name = re.sub('[^a-gA-G#-]+', '', n)
        if note_name:
            cur_note_list.append(note_name)
            
    cur_onset_list = list(set(cur_note_list) - set(prev_note_list))
    prev_note_list = cur_note_list
    
    
    note_vec = to_21d_vec(cur_note_list)
    onset_vec = to_21d_vec(cur_onset_list)
    this_onset_vec.extend(note_vec)
    this_onset_vec.extend(onset_vec)
    
    ######################################### Process beat position ########################################
    beat_pos = ''.join(row[["beat"]].values)
    meter = ''.join(row[["meter"]].values)
    meter = meter.replace("M", "")
    beat_vec = get_beat_vector(beat_pos, meter)
    this_onset_vec.extend(beat_vec)
    
    ######################################### Process chord label ########################################
    harm = ''.join(row[["harm"]].values)
    harm = re.sub('[()]', '', harm) # process string
    key = ''.join(row[["key"]].values)
    key_harm = key + ":" + harm
    #print(key_harm)
    try:
        chord_label = inputRN(key_harm)["Chord label"]
    except:
        continue # pass this onset slice if RN is not recognizable
    this_onset_vec.append(str(chord_label))
    #print(this_onset_vec)
  
    #########################################################################################################


Invalid entry. Try again. C#:Gr
Invalid entry. Try again. C#:Gr
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:Gr
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:Gr
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry. Try again. C#:C64
Invalid entry.

In [9]:
print(inputRN ("g:Cad64"))

{'Current key': 'G minor', 'Tonicized key': 'None', 'Intervallic construction': 'Gm3P5', 'Inversion': '2', 'Chord label': 'G minor', 'Default function': 'tonic', 'Chord pitches': "('G', 'Bb', 'D')", 'Chord pitch classes': '(7, 10, 2)'}
